In [1]:
import os
from langchain_community.document_loaders.wikipedia import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.llms.ollama import Ollama
from langchain.vectorstores.pgvector import PGVector
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [2]:
EMBEDDING_MODEL = "mxbai-embed-large"
LLM_MODEL = "gemma:2b"

def database_uri():
    user = os.getenv("POSTGRES_USER", "postgres")
    password = os.getenv("POSTGRES_PASSWORD", "")
    server = os.getenv("POSTGRES_SERVER", "localhost")
    port = os.getenv("POSTGRES_PORT", "5432")
    db = os.getenv("POSTGRES_DB", "localrag")
    return f"postgresql+psycopg2://{user}:{password}@{server}:{port}/{db}"

In [3]:
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
vector_db = PGVector(
    embedding_function=embeddings,
    connection_string=database_uri(),
    pre_delete_collection=True,
)
retriever = vector_db.as_retriever()
llm = Ollama(model=LLM_MODEL)

/Users/luis/.local/share/virtualenvs/local-rag-lip6Jf8R/lib/python3.12/site-packages/langchain_community/vectorstores/pgvector.py:293: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create adb migration for your metadata column to be JSONB and update your queries to use the new operators. 
  warn_deprecated(


In [4]:
def wikipedia_query(query: str):
    loader = WikipediaLoader(query=query, load_max_docs=3)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=80)
    texts = text_splitter.split_documents(docs)
    vector_db.add_documents(texts)

In [5]:
human_prompt = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.\n
Context: {context} \n
Question: {question} \n
Answer:
"""

prompt = ChatPromptTemplate.from_messages([("human", human_prompt)])

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
rag_chain.invoke("What is a transformer model?")

'A transformer model is a type of neural network architecture that uses the principles of transformer architecture to process sequential data. It is a powerful tool for tasks such as language translation, text summarization, and speech recognition.'

In [7]:
rag_chain.invoke("In what year was the Transformer paper published?")

"The provided context does not contain information regarding the year of the Transformer paper's publication, so I cannot answer this question from the provided context."

In [8]:
wikipedia_query("Transformer Architecture")
rag_chain.invoke("In what year was the Transformer paper published?")

'The Transformer paper was published in 2017.'

In [9]:
rag_chain.invoke("What is langchain?")

'The provided text does not contain information regarding langchain, so I cannot answer this question.'

In [10]:
wikipedia_query("Langchain")
rag_chain.invoke("What is langchain?")

'LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It is an open-source project that integrates with various cloud storage systems, APIs, and web services to facilitate the creation of applications using LLMs.'